<a href="https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/mid-one/mean_reversion/mean_reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![Cover](https://raw.githubusercontent.com/crunchdao/quickstarters/master/competitions/mid-one/cover.jpg)

# Mean Reversion

## How is it different to a forecast?

This notebook tries to predict whether a time-series will go up or down on average - though only when it has a strong opinion. To be precise, our attacker will consume a univariate sequence of numerical data points $x_1, x_2, \dots x_t$ and try to exploit deviations from the [martingale property](https://en.wikipedia.org/wiki/Martingale_(probability_theory)), which is to say that we expect the series $x_t$ to satisfy:

$$ E[x_{t+k}] \approx x_t $$

roughly. Of course, there's no such thing in this world as a perfect martingale and it is your job to indicate when

$$ E[x_{t+k}] > x_t + \epsilon $$

by returning a positive value, or conversely. Here $\epsilon$ finds interpretation as a trading cost. The attacker will *typically* return `0` meaning that it thinks:

$$  x_t - \epsilon   < E[x_{t+k}] < x_t + \epsilon $$

because trading opportunities are probably on the rare side - though obviously this is problem dependent. The $\epsilon$ and $k$ (`horizon`) parameters are set [here](https://github.com/microprediction/midone/blob/main/midone/gameconfig.py).

## Setup

In [ ]:
# Get a new token here: https://hub.crunchdao.com/competitions/mid-one/submit/via/notebook

%pip install --upgrade crunch-cli
!crunch setup --notebook mid-one hello --token aaaabbbbccccddddeeeeffff

## Imports

In [17]:
import statistics
import typing

import pandas

In [ ]:
import crunch

crunch = crunch.load_notebook()

### Loading data

In [ ]:
x_train, x_test = crunch.load_streams()

In [ ]:
print(f"Number of training stream: {len(x_train)}")

average_length = statistics.mean(len(stream) for stream in x_train)
print(f"Average length of training streams: {average_length}")

In [ ]:
def plot(stream):
    pandas.Series((
        message["x"]
        for message in stream
    )).plot()

plot(x_train[0])
plot(x_train[1])
plot(x_train[2])

## CrunchDAO Code Interface

[Submitting to the CrunchDAO platform requires 2 functions, `train` and `infer`.](https://docs.crunchdao.com/competitions/code-interface) Any line that is not in a function or is not an import will be commented when the notebook is processed.

The content of the function is the same as the example, but the train must save the model to be read in infer.

### The `train` function

In [11]:
def train(
    streams: typing.List[typing.Iterable[dict]],
    model_directory_path: str
):
    pass  # no training

## The `infer` function

Your notebook should have an infer function that can yield one prediction at a time.

In [ ]:
def infer(
    stream: typing.Iterator[dict],
    model_directory_path: str
):
    # Initialize your parameters.
    a = 0.15

    # Initialize your state.
    running_avg: float = None

    # Signals to the system that your attacker is initialized and ready.
    yield  # Leave this here.

    for message in stream:
        x = message["x"]

        # tick
        if running_avg is None:
            running_avg = x
        else:
            running_avg = (1 - a) * running_avg + a * x

        # Be sure to yield, even if the decision is zero.
        if x > running_avg + 2:
            yield -1  # sell
        elif x < running_avg - 2:
            yield 1  # buy
        else:
            yield 0  # hold


# A quick test that indicates how your infer function will be used when you upload this notebook:
messages = [{'x': 2.0}] * 10
for y in infer(messages, model_directory_path="resources/"):
    # the first value is `None`, this is intended
    print(y)

In [ ]:
prediction = crunch.test()
display(prediction)

display(prediction["prediction"].value_counts())

print("Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/mid-one/submit/via/notebook")